### Import Modules

In [1]:
# Python Libraries
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from torch.utils.data import DataLoader, TensorDataset
from tqdm.notebook import tqdm

In [2]:
# Our Modules
from modules.multihead_attention import MultiHeadAttention
from modules.data_preprocessing import DataPreprocessing
from modules.input import Input
from modules.encoder import Encoder
from modules.add_norm import AddNorm
from modules.ffn import FFN

### Parameters

In [3]:
data_path = fr"D:\courses\Transformers\Data\eng_french.csv"
max_sentence_length = 30
batch_size = 32
embedding_dim = 512
num_encoders = 1
num_multiheads = 8
encoder_hidden_layer_size = 2048

### Preprocess Data

In [4]:
preprocess = DataPreprocessing(
    data_path = data_path,
    max_sentence_length = max_sentence_length,
    batch_size = batch_size
)
dataset, dataloader = preprocess.forward()

Loading Data from D:\courses\Transformers\Data\eng_french.csv
Creating English vocabulary...
14117
Creating French vocabulary...
24185
Converting English Sentences to integer...
Converting French Sentences to integer...
Preparing Dataset for Input...
Done.


### Get Encoder Output

In [5]:
inp = Input(
    max_sentence_length,
    vocab_size = preprocess.english_vocab_size + 1,
    embedding_dim = embedding_dim
)

In [6]:
encoder = Encoder(
        batch_size,
        max_sentence_length,
        embedding_dim,
        num_multiheads,
        num_encoders,
        encoder_hidden_layer_size
)

In [ ]:
for batch in tqdm(dataloader):
    english_batch, french_batch = batch
        
    encoder_input = inp.forward(english_batch)

    encoder_output, query, key, value = encoder.forward(encoder_input)

  0%|          | 0/5489 [00:00<?, ?it/s]